In [1]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

In [2]:
# api_key 저장
api_key = 'RGAPI-b2bd2659-5fb4-4be0-821a-e3af369cc138' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

#### ranker name puuid CSV 파일의 puuid 사용해서 모든 puuid 의 최근 20게임 game id 추출, 테이블화

##### 만들었던 csv 파일 불러와서 df 에 저장

In [3]:
df = pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/ranker_name_puuid.csv')

In [4]:
# 불러온 테이블 확인
df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,puuid
0,GRANDMASTER,9sxIws6CI6fmK2sw2cAo_Xe8E82DNcR9XtHkWKC6W14SK2w,틈 빈,265,51,10,x5Aluxt-SEq_IUikKvH83sk0EkLXS7CP41bf66M_mBNdsiIUKYdavmHUPDKeWUBL6Ki5WwihBw2kQg
1,GRANDMASTER,BhJKOIvlHJRa_Ra0uNNDlrqN46i6snagNkH_p8VZR28GJA,Teukhan,327,84,55,Qd68F2UMKe7FMb-qpbUs3njG6k_4ZLwr2FRG0zHruSDr9XTtZOZ1cdAXyLd_kGAt0_IheL5QMQLdvQ
2,GRANDMASTER,80HtOC_fs-G1sNBcJFSlZXfNcYfz6he1pHYfBpJHkBs_2TM,COPY O,449,81,38,t_Wf-ad5Drr5FzTmxmkRnsEtBNUf-7PRoTll2k8kxN23sYTKJs2VOrOjeQ418PF31x2QQ2X02RrAtw
3,GRANDMASTER,pNUDUaztjVI1G7Jzoh4aVDMGr_8nDd0sMVeuWOvzwNMC6vB5,MessiOfTFT,394,86,53,4_-cZaXUQM6l7UQGnB69F9007HRO3dyTdKP-7_u0utPCg6YeZCBKMOZvoLFUdNyDa78Y7RBVqBXcGQ
4,GRANDMASTER,VVpwkqbuN3VVwG378diAEHea6jFrq1ADYBQAEstZizthf7k,이렇게 좋은 날,321,94,51,rx1nmyxEddDyQujL2nvjfiHV_LDcBEo-zyf16l3BFnXQExOMoB6tNss-WGlc_GRgcac2tg1GhDgBhg


##### puuid 를 넣으면 최근 20게임의 game id 를 반환하는 함수 생성

In [7]:
def get_20_gameId(puuid):

    # puuid 을 넣어줄 url 저장
    puuid_url = "https://Asia.api.riotgames.com/tft/" + f'match/v1/matches/by-puuid/{puuid}/ids?count=20'

    # puuid 넣었을 때, 반환값 저장
    gameId_list = requests.get(puuid_url, headers = request_header).json()

    # puuid 추출
    return gameId_list

In [9]:
# 함수 작동 테스트
get_20_gameId('x5Aluxt-SEq_IUikKvH83sk0EkLXS7CP41bf66M_mBNdsiIUKYdavmHUPDKeWUBL6Ki5WwihBw2kQg')[:3]

['KR_7004319615', 'KR_7004261207', 'KR_7004219924']

##### get_20_gameId 함수를 활용하여 모든 puuid 의 최근 20 게임 gameId 를 gameId 컬럼을 만들어 추가

In [18]:
# 위에서 정의한 함수를 사용하여 모든 puuid에 대한 게임 ID 목록을 가져와서 저장할 빈 리스트 생성
gameId_list = []

# 각 행의 puuid를 함수에 넣어 게임 ID 목록을 가져와서 리스트에 추가
for puuid in df['puuid']:
    gameId_list.append(get_20_gameId(puuid))

# 새로운 DataFrame인 gameId_df 생성하여 gameId_list를 해당 컬럼에 추가
gameId_df = pd.DataFrame({'gameId': gameId_list})

# 기존 DataFrame(df)와 gameId_df를 합쳐서 새로운 DataFrame인 final_df를 생성
final_df = pd.concat([df, gameId_df], axis=1)

In [23]:
# 티어와 게임 아이디만 남기고 모두 제거
final_df = final_df[['tier', 'gameId']]

final_df.head(3)

,tier,gameId
0,GRANDMASTER,"[KR_7004319615, KR_7004261207, KR_7004219924, KR_7004190074, KR_7004154411, KR_7004122827, KR_7003522190, KR_7003361432, KR_7003275706, KR_7003237553, KR_7001954405, KR_7001896892, KR_7001313513, KR_7001194313, KR_7001093815, KR_7000981008, KR_7000872942, KR_6999713328, KR_6999585116, KR_6999504198]"
1,GRANDMASTER,"[KR_7004321824, KR_7004287689, KR_7004251524, KR_7004212474, KR_7004186118, KR_7004159176, KR_7004130377, KR_7004111529, KR_7004082764, KR_7004069029, KR_7004041888, KR_7003813446, KR_7003330060, KR_7003267522, KR_7003217445, KR_7003176559, KR_7003137106, KR_7003082954, KR_7003045687, KR_7003019623]"
2,GRANDMASTER,"[KR_7004332795, KR_7004163795, KR_7004133580, KR_7003836228, KR_7003780848, KR_7003725466, KR_7003636575, KR_7003576286, KR_7003496678, KR_7003415677, KR_7003339946, KR_7003289547, KR_7003237553, KR_7003190560, KR_7003121826, KR_7003089848, KR_7003060002, KR_7003019623, KR_7003002191, KR_7002969109]"


##### 리스트로 되어있는 gameId 컬럼을 펼치고 중복값 제거하는 전처리 진행

In [26]:
# 리스트 형식으로 되어있는 gameId 컬럼 펼치기
final_df = final_df.explode('gameId')

final_df.head(3)

,tier,gameId
0,GRANDMASTER,KR_7004319615
0,GRANDMASTER,KR_7004261207
0,GRANDMASTER,KR_7004219924


In [27]:
# gameId 컬럼 값이 중복된 데이터 제거
final_df = final_df.drop_duplicates(subset=['gameId'])

final_df

,tier,gameId
0,GRANDMASTER,KR_7004319615
0,GRANDMASTER,KR_7004261207
0,GRANDMASTER,KR_7004219924
0,GRANDMASTER,KR_7004190074
0,GRANDMASTER,KR_7004154411
...,...,...
68,MASTER,KR_7002723663
68,MASTER,KR_7002712573
68,MASTER,KR_7002704044
68,MASTER,KR_7002678571


##### 전처리 완료한 테이블을 csv 파일로 저장

In [28]:
final_df.to_csv('tier_and_gameId.csv', index=False)

In [37]:
# 저장한 테이블 확인 (그랜드 마스터와 마스터가 섞여있는 큐는 그랜드 마스터 큐로 처리)
pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/tier_and_gameId.csv').head(193) ## 약 3분 1이 그마 큐

,tier,gameId
0,GRANDMASTER,KR_7004319615
1,GRANDMASTER,KR_7004261207
2,GRANDMASTER,KR_7004219924
3,GRANDMASTER,KR_7004190074
4,GRANDMASTER,KR_7004154411
...,...,...
188,GRANDMASTER,KR_7002604809
189,GRANDMASTER,KR_7002568008
190,GRANDMASTER,KR_7002507173
191,MASTER,KR_7004326801
